# About the dataset
Attribute Information:

1.	lettr	capital letter	(26 values from A to Z) 
2.	x-box	horizontal position of box	(integer) 
3.	y-box	vertical position of box	(integer) 
4.	width	width of box	(integer) 
5.	high height of box	(integer) 
6.	onpix	total # on pixels	(integer) 
7.	x-bar	mean x of on pixels in box	(integer) 
8.	y-bar	mean y of on pixels in box	(integer) 
9.	x2bar	mean x variance	(integer) 
10.	y2bar	mean y variance	(integer) 
11.	xybar	mean x y correlation	(integer) 
12.	x2ybr	mean of x * x * y	(integer) 
13.	xy2br	mean of x * y * y	(integer) 
14.	x-ege	mean edge count left to right	(integer) 
15.	xegvy	correlation of x-ege with y	(integer) 
16.	y-ege	mean edge count bottom to top	(integer) 
17.	yegvx	correlation of y-ege with x	(integer)

In [58]:
import pandas as pd

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data'
df = pd.read_csv(url, header=None)
df = df.sort_values([0]).rename(index=str, columns={0:"letter", 1:"x-pos", 2:"y-pos", 3:"width", 4:"height", 5:"numPixels", 6:"meanXPixels", 7:"meanYPixels", 8:"meanXVar", 9:"meanYVar", 10:"meanXYCor", 11:"meanXXY", 12:"meanXYY", 13:"meanEdgeCnt", 14:"xEdgCorr", 15:"meanYEdge", 16:"yEdgeCorr"})

def parseLetterToInt(x):
    return {
        'A': 1,
        'B': 2,
        'C': 3,
        'D': 4,
        'E': 5,
        'F': 6,
        'G': 7,
        'H': 8,
        'I': 9,
        'J': 10,
        'K': 11,
        'L': 12,
        'M': 13,
        'N': 14,
        'O': 15,
        'P': 16,
        'Q': 17,
        'R': 18,
        'S': 19,
        'T': 20,
        'U': 21,
        'V': 22,
        'W': 23,
        'X': 24,
        'Y': 25,
        'Z': 26
    }[x]

letterSeries = df.loc[:, "letter"]
print(letterSeries)
# for row in letterSeries:
#     row = parseLetterToInt(row)
#     print(row)

# for row in df.loc[:, "letter"]:
#     letterToInt = parseLetterToInt(row)
#     row = letterToInt
#     print(row)

# for index, row in df.iterrows():
# #     letter = row['letter']
# #     letterToInt = parseLetterToInt(letter)
# #     row['letter'] = letterToInt
    
#     range_max = max(row)
#     row = row / range_max
    
# # print(df.columns[0])

# print(df)


# num_df_inputs = len(df.columns[1:])
# print(num_df_inputs)

19999    A
1027     A
11108    A
1030     A
19420    A
7566     A
7582     A
11094    A
1041     A
1042     A
7608     A
15583    A
7612     A
15582    A
15575    A
7641     A
1062     A
11069    A
7752     A
13566    A
7749     A
7746     A
7696     A
7693     A
19423    A
7691     A
18083    A
1077     A
3360     A
7657     A
        ..
9793     Z
5643     Z
5647     Z
18929    Z
14287    Z
14289    Z
14297    Z
5685     Z
1882     Z
13144    Z
18953    Z
5728     Z
5910     Z
1859     Z
9756     Z
5749     Z
11622    Z
5771     Z
14313    Z
18972    Z
16628    Z
5823     Z
11627    Z
14331    Z
5843     Z
5868     Z
16594    Z
5906     Z
1853     Z
9712     Z
Name: letter, Length: 20000, dtype: object


In [ ]:
learning_rates = [0.5,0.1,0.01,0.001]
epochs = 10